In [1]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn import metrics

In [13]:
class CustomDataset:
    def __init__(self,data,targets):
        self.data = data
        self.targets = targets
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self,idx):
        current_sample = self.data[idx]
        current_target = self.targets[idx]
        
        return{
            "x": torch.tensor(current_sample),
            "y": torch.tensor(current_target)
        }    

In [14]:
data, targets = make_classification(n_samples=1000)

In [15]:
train_data,test_data, train_targets,test_targets = train_test_split(
   data,
   targets,
   stratify = targets
)

In [16]:
#making dataset object to load the training and testing data into memory 

training_dataset = CustomDataset(train_data, train_targets)
testing_dataset = CustomDataset(test_data, test_targets)

In [17]:
#making dataloader object to serve the dataset in batches

train_loader = torch.utils.data.DataLoader(training_dataset,
                                           batch_size = 4, num_workers = 2)

test_loader = torch.utils.data.DataLoader(testing_dataset, 
                                          batch_size = 4, num_workers = 2)

In [18]:
model = lambda x,W,b : torch.matmul(x,W) + b

In [19]:
W = torch.randn(20,1,requires_grad = True) # Adding weight
b = torch.randn(1,requires_grad =True) # Adding biases
W

tensor([[ 0.2009],
        [-0.8478],
        [-0.6741],
        [ 0.2303],
        [-1.3980],
        [-0.2925],
        [-0.2417],
        [ 0.2103],
        [-1.5155],
        [ 1.7513],
        [ 1.0913],
        [ 1.3428],
        [ 0.0895],
        [-0.3812],
        [ 1.2632],
        [-0.7757],
        [ 1.2971],
        [ 0.5324],
        [-0.4847],
        [ 0.2335]], requires_grad=True)

In [20]:
b

tensor([1.7132], requires_grad=True)

In [21]:
for data in train_loader :
    xtrain = data["x"]
    print (xtrain.shape)
    ytrain = data["y"]
    print (ytrain.shape)
    break

torch.Size([4, 20])
torch.Size([4])


In [22]:
for test_data in test_loader :
    xtest = test_data["x"]
    print (xtrain.shape)
    ytest = test_data["y"]
    print (ytrain.shape)
    break

torch.Size([4, 20])
torch.Size([4])


In [33]:
learning_rate = 0.001

for epoch in range(10):
    epoch_loss = 0
    counter = 0
    for data in train_loader :
#         print (data["x"])
        xtrain = data["x"]
        ytrain = data["y"]
        
        output = model(xtrain.float(),W,b)
        loss = torch.mean((ytrain.view(-1) - output.view(-1))**2)
        epoch_loss = epoch_loss + loss.item()
#         print(W.grad)
#         print(b.grad)
        loss.backward()
#         print(W.grad)
#         print(b.grad)
        #update weight and biases
        with torch.no_grad():
            W = W - learning_rate * W.grad
            b = b - learning_rate * b.grad
            
        W.requires_grad_(True)
        b.requires_grad_(True)
        counter += 1
        
        print(epoch,epoch_loss/counter)
        break

tensor([[-0.0439],
        [-0.0387],
        [ 0.0582],
        [ 0.1234],
        [ 0.4087],
        [-0.2243],
        [-0.3324],
        [-0.0238],
        [ 0.0288],
        [-0.0962],
        [-0.1870],
        [ 0.3588],
        [ 0.0627],
        [-0.1244],
        [ 0.0187],
        [-0.0120],
        [-0.1788],
        [-0.1710],
        [-0.2844],
        [ 0.0999]])
tensor([-0.1095])
0 0.019577698782086372
tensor([[-0.0427],
        [-0.0379],
        [ 0.0574],
        [ 0.1211],
        [ 0.4006],
        [-0.2204],
        [-0.3259],
        [-0.0238],
        [ 0.0286],
        [-0.0948],
        [-0.1835],
        [ 0.3525],
        [ 0.0615],
        [-0.1220],
        [ 0.0183],
        [-0.0125],
        [-0.1757],
        [-0.1687],
        [-0.2788],
        [ 0.0977]])
tensor([-0.1070])
1 0.018876170739531517
tensor([[-0.0414],
        [-0.0371],
        [ 0.0566],
        [ 0.1189],
        [ 0.3927],
        [-0.2167],
        [-0.3195],
        [-0.0237],
    

In [29]:
outputs = []
labels = []

with torch.no_grad():
    for data in test_loader:
        xtest = data["x"]
        ytest = data["y"]
        
        output = model(xtest.float(), W,b)
        labels.append(ytest)
        outputs.append(output)
        

In [30]:
metrics.roc_auc_score(torch.cat(labels).view(-1),torch.cat(outputs).view(-1))

0.964032